In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
from models import EMGConvNet

# === 1. Define Paths ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === 2. Load Data ===
X = np.load(X_path)   # shape: (5, 8, 230000)
y = np.load(y_path)

# === 3. Extract windows ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]  # label = pose at last time step
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === 4. Keep only sessions 0, 1, 2, 3 for cross-validation ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

# === 5. Define Dataset class ===
class EMGDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, standardize=True):
        self.standardize = standardize
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if self.standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# === 6. Perform 4-fold Cross-Validation ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for fold in range(4):
    print(f"\n===== Fold {fold+1} =====")
    
    # Build train/val split based on sessions
    X_train = np.vstack([X_sessions_cv[i] for i in range(4) if i != fold])
    y_train = np.vstack([y_sessions_cv[i] for i in range(4) if i != fold])
    
    X_val = X_sessions_cv[fold]
    y_val = y_sessions_cv[fold]
    
    train_dataset = EMGDataset(X_train, y_train)
    val_dataset = EMGDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)

    # Build a new model for each fold
    model = EMGConvNet(
        conv_layers_config=[(16, 5, 1), (32, 5, 2), (64, 3, 2)],
        fc_layers_config=[768, 512, 256, 128, 64],
        output_dim=51,
        verbose=False
    )
    model.build()
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    def compute_rmse(y_pred, y_true):
        return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

    for epoch in range(100):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)

        model.eval()
        val_loss = 0.0
        val_rmse = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                rmse = compute_rmse(outputs, y_batch)
                val_loss += loss.item() * X_batch.size(0)
                val_rmse += rmse.item() * X_batch.size(0)

        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        val_rmse /= len(val_loader.dataset)

        print(f"Epoch {epoch+1:2d} | Train MSE: {train_loss:.4f} | Val MSE: {val_loss:.4f} | Val RMSE: {val_rmse:.4f}")



===== Fold 1 =====
Epoch  1 | Train MSE: 241.7941 | Val MSE: 155.4830 | Val RMSE: 12.3920
Epoch  2 | Train MSE: 151.5756 | Val MSE: 160.7641 | Val RMSE: 12.6020
Epoch  3 | Train MSE: 145.2440 | Val MSE: 135.8437 | Val RMSE: 11.6113
Epoch  4 | Train MSE: 139.4539 | Val MSE: 151.6866 | Val RMSE: 12.2461
Epoch  5 | Train MSE: 143.7375 | Val MSE: 134.7240 | Val RMSE: 11.5748
Epoch  6 | Train MSE: 133.5207 | Val MSE: 136.7456 | Val RMSE: 11.6307
Epoch  7 | Train MSE: 97.5962 | Val MSE: 88.1232 | Val RMSE: 9.2507
Epoch  8 | Train MSE: 73.0904 | Val MSE: 68.9420 | Val RMSE: 8.2315
Epoch  9 | Train MSE: 63.2051 | Val MSE: 70.3549 | Val RMSE: 8.2774
Epoch 10 | Train MSE: 55.3050 | Val MSE: 66.1819 | Val RMSE: 8.0689
Epoch 11 | Train MSE: 56.2595 | Val MSE: 61.7439 | Val RMSE: 7.7851
Epoch 12 | Train MSE: 52.2481 | Val MSE: 61.9122 | Val RMSE: 7.7945
Epoch 13 | Train MSE: 51.6163 | Val MSE: 65.6466 | Val RMSE: 8.0220
Epoch 14 | Train MSE: 52.2411 | Val MSE: 63.3619 | Val RMSE: 7.8810
Epoch 15 |

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
from pathlib import Path
from models import EMGConvNet

# === 1. Define Paths ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === 2. Load Data ===
X = np.load(X_path)  # shape: (5, 8, 230000)
y = np.load(y_path)

# === 3. Extract windows by session ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]  # label = pose at last time step
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === 4. Keep only sessions 0,1,2,3 for cross-validation ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

# === 5. Define Dataset class ===
class EMGDataset(Dataset):
    def __init__(self, X, y, standardize=True):
        self.standardize = standardize
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if self.standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# === 6. Define architecture config separately ===
architecture_config = {
    "conv_layers": [(16, 5, 1), (32, 5, 2), (64, 3, 2)],
    "fc_layers": [768, 512, 256, 128, 64],
    "conv_dropouts": [0, 0, 0],
    "fc_dropouts": [0, 0, 0, 0, 0]
}

experiment_log = {
    "architecture": architecture_config,
    "folds": []
}

# === 7. Perform 4-fold Cross-Validation ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for fold_idx in range(4):
    print(f"\n===== Fold {fold_idx+1} =====")
    fold_log = {"fold_number": fold_idx, "epochs": []}

    # Build train/val split based on sessions
    X_train = np.vstack([X_sessions_cv[i] for i in range(4) if i != fold_idx])
    y_train = np.vstack([y_sessions_cv[i] for i in range(4) if i != fold_idx])
    X_val = X_sessions_cv[fold_idx]
    y_val = y_sessions_cv[fold_idx]

    train_dataset = EMGDataset(X_train, y_train)
    val_dataset = EMGDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)

    model = EMGConvNet(
        conv_layers_config=architecture_config["conv_layers"],
        fc_layers_config=architecture_config["fc_layers"],
        conv_dropouts=architecture_config["conv_dropouts"],
        fc_dropouts=architecture_config["fc_dropouts"],
        output_dim=51,
        verbose=False
    )
    model.build()
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    def compute_rmse(y_pred, y_true):
        return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

    for epoch in range(100):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)

        model.eval()
        val_loss = 0.0
        val_rmse = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                rmse = compute_rmse(outputs, y_batch)
                val_loss += loss.item() * X_batch.size(0)
                val_rmse += rmse.item() * X_batch.size(0)

        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        val_rmse /= len(val_loader.dataset)

        epoch_log = {
            "epoch": epoch + 1,
            "train_mse": train_loss,
            "val_mse": val_loss,
            "val_rmse": val_rmse
        }
        fold_log["epochs"].append(epoch_log)

        print(f"Epoch {epoch+1:2d} | Train MSE: {train_loss:.4f} | Val MSE: {val_loss:.4f} | Val RMSE: {val_rmse:.4f}")

    experiment_log["folds"].append(fold_log)

# === 8. Save experiment log ===

log_path = Path("experiment_log.json")

if log_path.exists():
    with open(log_path, "r") as f:
        existing_data = json.load(f)
else:
    existing_data = []

existing_data.append(experiment_log)

with open(log_path, "w") as f:
    json.dump(existing_data, f, indent=4)



===== Fold 1 =====
Epoch  1 | Train MSE: 270.1534 | Val MSE: 195.7387 | Val RMSE: 13.8879
Epoch  2 | Train MSE: 157.6006 | Val MSE: 183.0145 | Val RMSE: 13.4642
Epoch  3 | Train MSE: 149.2386 | Val MSE: 141.4410 | Val RMSE: 11.8324
Epoch  4 | Train MSE: 140.5110 | Val MSE: 134.3813 | Val RMSE: 11.5557
Epoch  5 | Train MSE: 138.6086 | Val MSE: 160.2620 | Val RMSE: 12.5875
Epoch  6 | Train MSE: 139.8710 | Val MSE: 143.3964 | Val RMSE: 11.9104
Epoch  7 | Train MSE: 117.8616 | Val MSE: 93.7630 | Val RMSE: 9.6233
Epoch  8 | Train MSE: 72.7861 | Val MSE: 68.3536 | Val RMSE: 8.1907
Epoch  9 | Train MSE: 63.3437 | Val MSE: 63.1300 | Val RMSE: 7.8655
Epoch 10 | Train MSE: 54.1293 | Val MSE: 67.7624 | Val RMSE: 8.1257
Epoch 11 | Train MSE: 57.4323 | Val MSE: 82.0178 | Val RMSE: 8.8900
Epoch 12 | Train MSE: 60.2490 | Val MSE: 60.6463 | Val RMSE: 7.7070
Epoch 13 | Train MSE: 53.0669 | Val MSE: 64.3633 | Val RMSE: 7.9419
Epoch 14 | Train MSE: 53.3276 | Val MSE: 67.4614 | Val RMSE: 8.0937
Epoch 15 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
from pathlib import Path
from models import EMGConvNet, TrainingManager, CrossValidationManager
from utils import save_experiment_log

In [78]:
# === Paths to data ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === Load Data ===
X = np.load(X_path)   # (5, 8, 230000)
y = np.load(y_path)   # (5, 51, 230000)

# === Extract windows by session ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === Keep only first 4 sessions for CV ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

class EMGDataset(Dataset):
    def __init__(self, X, y, standardize=True):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [3]:
def save_experiment_log(log, path="logs/experiment_log.json"):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)

    if path.exists():
        with open(path, "r") as f:
            existing = json.load(f)
    else:
        existing = []

    existing.append(log)

    with open(path, "w") as f:
        json.dump(existing, f, indent=4)


In [4]:
# Model and training configs
model_config = {
    "conv_layers_config": [(16, 5, 1), (32, 5, 2)],
    "fc_layers_config": [512, 256, 128, 64],
    "output_dim": 51,
    "verbose": False
}

training_config = {
    "lr": 1e-3,
    "epochs": 100,
    "batch_size": 64,
    "log_every": 1
}

# Cross-validation runner
cross_validator = CrossValidationManager(
    model_class=EMGConvNet,
    model_config=model_config,
    data=X_sessions_cv,
    labels=y_sessions_cv,
    training_config=training_config,
    dataset_class=EMGDataset,
    dataset_config={"standardize": True},
    n_folds=4
)

# Run and save
experiment_log = cross_validator.run()
save_experiment_log(experiment_log, path="logs/neuralnetwork_log.json")



===== Fold 1/4 =====
Epoch   1 | Train MSE: 260.1710 | Val MSE: 171.0860 | Val RMSE: 12.9763
Epoch   2 | Train MSE: 163.4468 | Val MSE: 152.0605 | Val RMSE: 12.2575
Epoch   3 | Train MSE: 150.4566 | Val MSE: 143.7922 | Val RMSE: 11.9183
Epoch   4 | Train MSE: 113.1277 | Val MSE: 96.6523 | Val RMSE: 9.7362
Epoch   5 | Train MSE: 77.1006 | Val MSE: 79.9829 | Val RMSE: 8.8508
Epoch   6 | Train MSE: 65.5053 | Val MSE: 90.2205 | Val RMSE: 9.3425
Epoch   7 | Train MSE: 59.6206 | Val MSE: 77.8296 | Val RMSE: 8.7337
Epoch   8 | Train MSE: 59.6210 | Val MSE: 87.4416 | Val RMSE: 9.1830
Epoch   9 | Train MSE: 58.8816 | Val MSE: 71.5518 | Val RMSE: 8.4012
Epoch  10 | Train MSE: 59.7890 | Val MSE: 69.7020 | Val RMSE: 8.3023
Epoch  11 | Train MSE: 55.9174 | Val MSE: 69.9906 | Val RMSE: 8.2959
Epoch  12 | Train MSE: 53.5833 | Val MSE: 65.7622 | Val RMSE: 8.0393
Epoch  13 | Train MSE: 54.0129 | Val MSE: 64.5510 | Val RMSE: 7.9809
Epoch  14 | Train MSE: 53.4739 | Val MSE: 61.6555 | Val RMSE: 7.7989
Ep

In [5]:
import itertools

def generate_architecture_configs(
    conv_layer_depths=(2, 3),
    conv_out_channels=(16, 32, 64),
    conv_kernel_sizes=(3, 5),
    conv_strides=(1, 2),
    fc_layer_depths=(1, 2, 3),
    fc_layer_sizes=(128, 256, 512),
    dropout_levels=(0.0, 0.05, 0.1),
    output_dim=51,
    verbose=False
):
    configs = []

    # Cartesian product of all conv layer configurations per layer
    conv_options = list(itertools.product(conv_out_channels, conv_kernel_sizes, conv_strides))

    for conv_depth in conv_layer_depths:
        for conv_layer_combo in itertools.product(conv_options, repeat=conv_depth):
            for fc_depth in fc_layer_depths:
                for fc_layer_combo in itertools.product(fc_layer_sizes, repeat=fc_depth):
                    for dropout in dropout_levels:
                        config = {
                            "conv_layers_config": list(conv_layer_combo),
                            "fc_layers_config": list(fc_layer_combo),
                            "conv_dropouts": [dropout] * conv_depth,
                            "fc_dropouts": [dropout] * fc_depth,
                            "output_dim": output_dim,
                            "verbose": verbose
                        }
                        configs.append(config)

    return configs


In [30]:
configs = generate_architecture_configs(
    conv_layer_depths=[2],
    conv_out_channels=[32, 64],
    conv_kernel_sizes=[3, 5],
    conv_strides=[1],
    fc_layer_depths=[4],
    fc_layer_sizes=[(i+1)*64 for i in range(4)],
    dropout_levels=[0.0, 0.05, 0.1]
)

print(f"Generated {len(configs)} architecture configurations.")


Generated 12288 architecture configurations.


In [34]:
configs[1230]

{'conv_layers_config': [(32, 3, 1), (32, 5, 1)],
 'fc_layers_config': [192, 128, 192, 192],
 'conv_dropouts': [0.0, 0.0],
 'fc_dropouts': [0.0, 0.0, 0.0, 0.0],
 'output_dim': 51,
 'verbose': False}

In [75]:
def generate_structured_architectures(
    output_dim=51,
    dropout_levels=(0.0, 0.05, 0.1),
    conv_templates=None,
    fc_templates=None,
    max_conv_depth=None,
    max_fc_depth=None,
    verbose=False
):
    configs = []

    # --- Default Conv Families ---
    default_conv_templates = {
        "shallow_wide": [[(64, 5, 1)], [(128, 5, 1), (128, 3, 1)]],
        "deep_narrow": [[(32, 3, 1)] * 4, [(64, 3, 1)] * 5],
        "expanding": [[(32, 5, 1), (64, 3, 1), (128, 3, 1)]],
        "bottleneck": [[(128, 5, 1), (64, 3, 1), (32, 3, 1)]],
        "oscillating": [[(64, 5, 1), (128, 3, 1), (64, 3, 1)]],
    }

    default_fc_templates = {
        "flat_wide": [[512], [512, 512]],
        "shrinking": [[512, 256], [256, 128]],
        "expanding": [[128, 256, 512]],
        "bottleneck": [[512, 128, 256]],
        "oscillating": [[256, 128, 256]],
    }

    conv_templates = conv_templates or default_conv_templates
    fc_templates = fc_templates or default_fc_templates

    for conv_name, conv_list in conv_templates.items():
        for fc_name, fc_list in fc_templates.items():
            for conv_cfg in conv_list:
                if max_conv_depth is not None and len(conv_cfg) > max_conv_depth:
                    continue
                for fc_cfg in fc_list:
                    if max_fc_depth is not None and len(fc_cfg) > max_fc_depth:
                        continue
                    for d in dropout_levels:
                        config = {
                            "conv_layers_config": list(conv_cfg),
                            "fc_layers_config": list(fc_cfg),
                            "conv_dropouts": [d] * len(conv_cfg),
                            "fc_dropouts": [d] * len(fc_cfg),
                            "output_dim": output_dim,
                            "verbose": verbose,
                            "conv_family": conv_name,
                            "fc_family": fc_name,
                            "dropout_level": d
                        }
                        configs.append(config)

    return configs



def make_fc_templates_sampled(
    widths=(128, 256, 512, 768, 1024, 1536, 2048),
    depths=(2, 3, 4, 5),
    shapes=("increasing", "decreasing", "symmetric", "flat"),
    limit_per_shape=10
):
    import random
    fc_templates = {}

    def generate_shape(shape, depth, widths):
        if shape == "increasing":
            return sorted(random.sample(widths, depth))
        elif shape == "decreasing":
            return sorted(random.sample(widths, depth), reverse=True)
        elif shape == "symmetric":
            half = sorted(random.sample(widths, depth // 2))
            return half + half[::-1] if depth % 2 == 0 else half + [random.choice(widths)] + half[::-1]
        elif shape == "flat":
            w = random.choice(widths)
            return [w] * depth

    for shape in shapes:
        for depth in depths:
            key = f"{shape}_d{depth}"
            fc_templates[key] = []
            for _ in range(limit_per_shape):
                template = generate_shape(shape, depth, widths)
                fc_templates[key].append(template)

    return fc_templates



fc_templates = make_fc_templates_sampled(
    widths=(128, 256, 512, 768, 1024, 1536, 2048),
    depths=(2, 3, 4, 5),
    shapes=("increasing", "decreasing", "symmetric", "flat"),
    limit_per_shape=5  # small for now; increase as needed
)

configs = generate_structured_architectures(
    fc_templates=fc_templates,
    dropout_levels=[0.0, 0.05],
    max_fc_depth=5,
    max_conv_depth=3
)


In [84]:
configs[1]

{'conv_layers_config': [(64, 5, 1)],
 'fc_layers_config': [256, 1024],
 'conv_dropouts': [0.05],
 'fc_dropouts': [0.05, 0.05],
 'output_dim': 51,
 'verbose': False,
 'conv_family': 'shallow_wide',
 'fc_family': 'increasing_d2',
 'dropout_level': 0.05}

In [69]:
import random
from itertools import product

class ArchitectureSampler:
    def __init__(
        self,
        widths=(128, 256, 512, 768, 1024, 1536, 2048),
        depths=(2, 3, 4, 5),
        shapes=("increasing", "decreasing", "symmetric", "flat"),
        limit_per_shape=10,
        seed=None
    ):
        self.widths = widths
        self.depths = depths
        self.shapes = shapes
        self.limit = limit_per_shape
        self.seed = seed
        if seed is not None:
            random.seed(seed)

    def _generate_shape(self, shape, depth):
        if shape == "increasing":
            return sorted(random.sample(self.widths, depth))
        elif shape == "decreasing":
            return sorted(random.sample(self.widths, depth), reverse=True)
        elif shape == "symmetric":
            half = sorted(random.sample(self.widths, depth // 2))
            return half + half[::-1] if depth % 2 == 0 else half + [random.choice(self.widths)] + half[::-1]
        elif shape == "flat":
            w = random.choice(self.widths)
            return [w] * depth
        else:
            raise ValueError(f"Unknown shape type: {shape}")

    def generate_fc_templates(self):
        fc_templates = {}
        for shape in self.shapes:
            for depth in self.depths:
                key = f"{shape}_d{depth}"
                fc_templates[key] = [
                    self._generate_shape(shape, depth)
                    for _ in range(self.limit)
                ]
        return fc_templates


In [70]:
sampler = ArchitectureSampler(
    widths=(128, 256, 512, 1024, 2048),
    depths=(2, 3, 4, 5),
    limit_per_shape=5,
    seed=42
)

fc_templates = sampler.generate_fc_templates()

configs = generate_structured_architectures(
    fc_templates=fc_templates,
    dropout_levels=[0.0, 0.05],
    max_fc_depth=5
)


In [77]:
len(configs)

800

In [79]:
import uuid
import os
import traceback
from models import EMGConvNet  # delay import for process safety
from concurrent.futures import ProcessPoolExecutor, as_completed
import random


# === Paths to data ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === Load Data ===
X = np.load(X_path)   # (5, 8, 230000)
y = np.load(y_path)   # (5, 51, 230000)

# === Extract windows by session ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === Keep only first 4 sessions for CV ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

class EMGDataset(Dataset):
    def __init__(self, X, y, standardize=True):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# def run_experiment(config, base_output_dir="logs_parallel"):


#     try:
#         os.makedirs(base_output_dir, exist_ok=True)
#         config_id = str(uuid.uuid4())[:8]

#         cv = CrossValidationManager(
#             model_class=EMGConvNet,
#             model_config=config,
#             data=X_sessions_cv,
#             labels=y_sessions_cv,
#             training_config={
#                 "lr": 1e-3,
#                 "epochs": 100,
#                 "batch_size": 64,
#                 "log_every": 1
#             },
#             dataset_class=EMGDataset,
#             dataset_config={"standardize": True},
#             n_folds=4
#         )

#         log = cv.run()

#         # Attach config ID for traceability
#         log["experiment_id"] = config_id

#         # Save to individual file
#         out_path = os.path.join(base_output_dir, f"experiment_{config_id}.json")
#         with open(out_path, "w") as f:
#             import json
#             json.dump(log, f, indent=4)

#         return f"✔ Config {config_id} complete."

#     except Exception as e:
#         print(f"✘ Config failed:\n{traceback.format_exc()}")
#         return f"✘ Error: {e}"


# def run_all_parallel(configs, max_workers=2):
#     results = []
#     with ProcessPoolExecutor(max_workers=max_workers) as executor:
#         futures = [executor.submit(run_experiment, cfg) for cfg in configs]
#         for f in as_completed(futures):
#             result = f.result()
#             print(result)
#             results.append(result)
#     return results



In [82]:


# Subsample randomly (optional)
sampled_configs = random.sample(configs, 10)  # adjust size as needed

# Run them in parallel
run_all_parallel(sampled_configs[:2], max_workers=1)


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [102]:
class ArchitectureSampler:
    def __init__(
        self,
        widths=(128, 256, 512, 768, 1024, 1536),
        depths=(2, 3, 4, 5),
        shapes=("increasing", "decreasing", "symmetric", "flat"),
        limit_per_shape=10,
        dropout_levels=(0.0, 0.05, 0.1),
        max_fc_depth=None,
        max_conv_depth=None,
        conv_templates=None,
        output_dim=51,
        seed=None,
        verbose=False
    ):
        self.widths = widths
        self.depths = depths
        self.shapes = shapes
        self.limit = limit_per_shape
        self.dropout_levels = dropout_levels
        self.max_fc_depth = max_fc_depth
        self.max_conv_depth = max_conv_depth
        self.output_dim = output_dim
        self.verbose = verbose
        self.conv_templates = conv_templates
        self.seed = seed

        if seed is not None:
            random.seed(seed)

        self._configs = []

    def _generate_shape(self, shape, depth):
        if shape == "increasing":
            return sorted(random.sample(self.widths, depth))
        elif shape == "decreasing":
            return sorted(random.sample(self.widths, depth), reverse=True)
        elif shape == "symmetric":
            half = sorted(random.sample(self.widths, depth // 2))
            return half + half[::-1] if depth % 2 == 0 else half + [random.choice(self.widths)] + half[::-1]
        elif shape == "flat":
            w = random.choice(self.widths)
            return [w] * depth
        else:
            raise ValueError(f"Unknown shape type: {shape}")

    def generate_fc_templates(self):
        fc_templates = {}
        for shape in self.shapes:
            for depth in self.depths:
                key = f"{shape}_d{depth}"
                fc_templates[key] = [
                    self._generate_shape(shape, depth)
                    for _ in range(self.limit)
                ]
        return fc_templates

    def generate_configs(self):
        fc_templates = self.generate_fc_templates()

        # fallback if no conv templates provided
        conv_templates = self.conv_templates or {
            "shallow_wide": [[(64, 5, 1)], [(128, 5, 1), (128, 3, 1)]],
            "deep_narrow": [[(32, 3, 1)] * 4, [(64, 3, 1)] * 5],
            "expanding": [[(32, 5, 1), (64, 3, 1), (128, 3, 1)]],
            "bottleneck": [[(128, 5, 1), (64, 3, 1), (32, 3, 1)]],
            "oscillating": [[(64, 5, 1), (128, 3, 1), (64, 3, 1)]],
        }

        configs = []

        for conv_name, conv_list in conv_templates.items():
            for fc_name, fc_list in fc_templates.items():
                for conv_cfg in conv_list:
                    if self.max_conv_depth and len(conv_cfg) > self.max_conv_depth:
                        continue
                    for fc_cfg in fc_list:
                        if self.max_fc_depth and len(fc_cfg) > self.max_fc_depth:
                            continue
                        for d in self.dropout_levels:
                            config = {
                                "conv_layers_config": list(conv_cfg),
                                "fc_layers_config": list(fc_cfg),
                                "conv_dropouts": [d] * len(conv_cfg),
                                "fc_dropouts": [d] * len(fc_cfg),
                                "output_dim": self.output_dim,
                                "verbose": self.verbose,
                                "conv_family": conv_name,
                                "fc_family": fc_name,
                                "dropout_level": d
                            }
                            configs.append(config)

        self._configs = configs
        return configs
    
    def shuffle_configs(self, seed=None):
        if seed is not None:
            random.seed(seed)
        random.shuffle(self._configs)

    def get_all_configs(self):
        return self._configs

    def get_model_configs(self):
        keys = ["conv_layers_config", "fc_layers_config", "conv_dropouts", "fc_dropouts", "output_dim", "verbose"]
        return [{k: c[k] for k in keys} for c in self._configs]


In [107]:
conv_templates = {
    "shallow_narrow": [[(16, 5, 1)], [(32, 5, 1), (32, 3, 1)]],
    "moderate": [[(16, 5, 1), (32, 3, 1)]],
    "expanding_light": [[(16, 5, 1), (32, 3, 1), (64, 3, 1)]],
    "bottleneck_light": [[(64, 5, 1), (32, 3, 1), (16, 3, 1)]],
    "oscillating_light": [[(32, 5, 1), (64, 3, 1), (32, 3, 1)]],
}

sampler = ArchitectureSampler(limit_per_shape=3, seed=42, conv_templates=conv_templates)
sampler.generate_configs()
sampler.shuffle_configs(seed=42)

configs = sampler.get_all_configs()
print("All configs (with metadata):", len(sampler.get_all_configs()))
print("Clean model configs:", len(sampler.get_model_configs()))

# For parallel training:
clean_configs = sampler.get_model_configs()


All configs (with metadata): 864
Clean model configs: 864


In [109]:
clean_configs[832]

{'conv_layers_config': [(32, 5, 1), (32, 3, 1)],
 'fc_layers_config': [1024, 1024],
 'conv_dropouts': [0.0, 0.0],
 'fc_dropouts': [0.0, 0.0],
 'output_dim': 51,
 'verbose': False}

In [111]:
# Model and training configs

training_config = {
    "lr": 2e-3,
    "epochs": 100,
    "batch_size": 256,
    "log_every": 1
}

for config in clean_configs:
# Cross-validation runner
    cross_validator = CrossValidationManager(
    model_class=EMGConvNet,
    model_config=config,
    data=X_sessions_cv,
    labels=y_sessions_cv,
    training_config=training_config,
    dataset_class=EMGDataset,
    dataset_config={"standardize": True},
    n_folds=4
)
    # Run and save
    experiment_log = cross_validator.run()
    save_experiment_log(experiment_log, path="logs/neuralnetwork_log.json")


===== Fold 1/4 =====
Epoch   1 | Train MSE: 327.0008 | Val MSE: 308.9979 | Val RMSE: 17.5668
Epoch   2 | Train MSE: 187.2818 | Val MSE: 189.4766 | Val RMSE: 13.7503
Epoch   3 | Train MSE: 154.7449 | Val MSE: 172.5621 | Val RMSE: 13.1257
Epoch   4 | Train MSE: 138.3576 | Val MSE: 184.4517 | Val RMSE: 13.5703
Epoch   5 | Train MSE: 102.6844 | Val MSE: 145.1044 | Val RMSE: 12.0282
Epoch   6 | Train MSE: 76.7019 | Val MSE: 71.4161 | Val RMSE: 8.4385
Epoch   7 | Train MSE: 64.4216 | Val MSE: 81.0476 | Val RMSE: 8.9738
Epoch   8 | Train MSE: 61.7917 | Val MSE: 65.8257 | Val RMSE: 8.0990
Epoch   9 | Train MSE: 55.6343 | Val MSE: 63.9347 | Val RMSE: 7.9849
Epoch  10 | Train MSE: 53.8505 | Val MSE: 64.7353 | Val RMSE: 8.0325
Epoch  11 | Train MSE: 50.5719 | Val MSE: 61.0009 | Val RMSE: 7.7991
Epoch  12 | Train MSE: 48.1453 | Val MSE: 61.0059 | Val RMSE: 7.7999
Epoch  13 | Train MSE: 44.2628 | Val MSE: 54.5870 | Val RMSE: 7.3762
Epoch  14 | Train MSE: 39.9701 | Val MSE: 53.8606 | Val RMSE: 7.32

In [106]:
clean_configs[0]

{'conv_layers_config': [(128, 5, 1), (128, 3, 1)],
 'fc_layers_config': [1024, 128],
 'conv_dropouts': [0.05, 0.05],
 'fc_dropouts': [0.05, 0.05],
 'output_dim': 51,
 'verbose': False}

In [46]:
from utils import ExperimentSelector

selector = ExperimentSelector(r"D:\Uni\F422\pose-estimation-from-emg-signal-team-1\logs\neuralnetwork_log.json")

# Get top 20 by final RMSE, but filter for low variance
top_stable = selector.select(sort_by="final_avg", top_n=20, max_variance=0.15, return_full=True)

# Get top 20 by convergence potential
top_converging = selector.select(sort_by="potential", top_n=20)

# Get architectures that reached the lowest point anywhere
top_by_lowest = selector.select(sort_by="lowest_point", top_n=20)

# Get low-bias models (slow starters, strong finishers)
top_bias = selector.select(sort_by="bias", top_n=20)


In [48]:
from utils import save_experiment_log

save_experiment_log(top_stable, r"logs\stable_results.json")

In [51]:
models_summary = selector.select(sort_by="final_avg", top_n=20, return_full=True)
save_experiment_log(models_summary, r"logs\stable_results.json")

In [55]:
models_summary = selector.select(sort_by="final_avg", top_n=20)


In [57]:
# models_summary
save_experiment_log(models_summary, r"logs\summarised_stable_results.json")